In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys
import time
import sklearn
from tensorflow import keras

In [3]:
# index
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
print(t)
print(t[:, 1:])
print(t[..., 1])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[2. 3.]
 [5. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


In [4]:
# op
print(t + 10)
print(t -1)
print(tf.square(t))
print(t @ tf.transpose(t))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[0. 1. 2.]
 [3. 4. 5.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[14. 32.]
 [32. 77.]], shape=(2, 2), dtype=float32)


In [6]:
# numpy conversion
print(t.numpy())
print(np.square(t))
t_np = [[1., 2., 3.], [4., 5., 6.]]
print(tf.constant(t_np))
t2_np = np.array([[1., 2., 3.], [4., 5., 6.]])
print(tf.constant(t2_np))

[[1. 2. 3.]
 [4. 5. 6.]]
[[ 1.  4.  9.]
 [16. 25. 36.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)


In [7]:
# Scalars:  0维
t = tf.constant(2.51)
print(t.numpy())
print(t.shape)

2.51
()


In [9]:
# strings
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)


In [11]:
# string array
ta = tf.constant(["cafe", "coffee", "咖啡"])
print(ta)
print(tf.strings.length(ta, unit="UTF8_CHAR"))
r = tf.strings.unicode_decode(ta, "UTF8")
print(r)

tf.Tensor([b'cafe' b'coffee' b'\xe5\x92\x96\xe5\x95\xa1'], shape=(3,), dtype=string)
tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


In [13]:
# ragged tensor
t = tf.ragged.constant([[21, 22], [31, 32, 33], [], [15]])
print(t)
print(t[0])
print(t[1:2])  # [1, 2)


<tf.RaggedTensor [[21, 22], [31, 32, 33], [], [15]]>
tf.Tensor([21 22], shape=(2,), dtype=int32)
<tf.RaggedTensor [[31, 32, 33]]>


In [25]:
# ops on ragged tensor

r2 = tf.ragged.constant([[41, 42, 43], [16], []])
print(tf.concat([t, r2], axis = 0))
print(tf.concat([t, r2], axis = 1))

<tf.RaggedTensor [[21, 22], [31, 32, 33], [], [15], [41, 42, 43], [16], []]>


InvalidArgumentError: Input tensors have incompatible shapes.
Condition x == y did not hold.
First 1 elements of x:
[3]
First 1 elements of y:
[4]


In [21]:
r3 = tf.ragged.constant([[41, 42, 43], [16], [], [24, 28]])
print(tf.concat([t, r3], axis=1))

<tf.RaggedTensor [[21, 22, 41, 42, 43], [31, 32, 33, 16], [], [15, 24, 28]]>


In [22]:
print(t.to_tensor())

tf.Tensor(
[[21 22  0]
 [31 32 33]
 [ 0  0  0]
 [15  0  0]], shape=(4, 3), dtype=int32)


In [30]:
# SparseTensor
s = tf.SparseTensor(indices = [[0, 1], [1, 0], [2, 3]],
                  values=[3, 5, 6],
                  dense_shape=[3, 4])
print(s)
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([3 5 6], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0 3 0 0]
 [5 0 0 0]
 [0 0 0 6]], shape=(3, 4), dtype=int32)


In [32]:
# ops on SparseTensor
s1 = s * 2
try:
    s2 = s + 1
except TypeError as ex:
    print(ex)
    
s3 = tf.constant([[1, 2], 
                            [2, 3],
                            [3, 4],
                            [4, 5]])
s4 = tf.sparse.sparse_dense_matmul(s, s3)
print(s4)

unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 6  9]
 [ 5 10]
 [24 30]], shape=(3, 2), dtype=int32)


In [33]:
# SparseTensor:  indices require order
s = tf.SparseTensor(indices = [[2, 1], [1, 0], [2, 3]],
                  values=[3, 5, 6],
                  dense_shape=[3, 4])
print(s)
try:
    print(tf.sparse.to_dense(s))
except TypeError as ex:
    print(ex)


SparseTensor(indices=tf.Tensor(
[[2 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([3 5 6], shape=(3,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))


InvalidArgumentError: indices[1] = [1,0] is out of order [Op:SparseToDense]

In [34]:
s6 = tf.sparse.reorder(s)
print(tf.sparse.to_dense(s6))

tf.Tensor(
[[0 0 0 0]
 [5 0 0 0]
 [0 3 0 6]], shape=(3, 4), dtype=int32)


In [35]:
# Variable
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
print(v.numpy())
print(v.value())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
[[1. 2. 3.]
 [4. 5. 6.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


In [36]:
v.assign(2.*v)
print(v.numpy())
v[0, 1].assign(41)
print(v.numpy())
v[1].assign([6., 7., 8.])
print(v.numpy())

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[ 2. 41.  6.]
 [ 8. 10. 12.]]
[[ 2. 41.  6.]
 [ 6.  7.  8.]]


In [37]:
try:
    v[1] = [4., 5., 6.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment
